# Backward Elimination vs. Backward Elimination + Ridge vs. Lasso

In [ ]:
!pip install statsmodels

In [ ]:
from google.colab import drive
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn import  linear_model
import statsmodels.api as sm

drive.mount('/content/gdrive')

train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/test.csv")
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')
submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sample_submission.csv')

In [30]:
data = train

data1 = data.copy().fillna(0)
feature = data1.drop_duplicates('단지코드').sort_values('단지코드').set_index('단지코드')
feature['주거세대수'] = feature['총세대수'] - feature['공가수']

data2 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data2['총전용면적'] = data2['전용면적'] * data2['전용면적별세대수']
data2 = data2.groupby('단지코드')['총전용면적'].sum().reset_index().set_index('단지코드')

data3 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data3['전용면적'] = np.round(data3['전용면적'], -1).astype(int)
list2 = []
for i in range(10, 610, 10):
  list2.append('면적' + str(i))
  data3['면적' + str(i)] = (data3['전용면적'] == i) * data3['전용면적별세대수']
data3 = data3.groupby('단지코드')[list2].sum().reset_index().set_index('단지코드')

feature = pd.merge(feature, data2, on = '단지코드')
feature = pd.merge(feature, data3, on = '단지코드')
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수']
list1 = []
for i in train['지역'].unique():
  list1.append(i + '여부')
  feature[i + '여부'] = (feature['지역'] == i) * 1

feature['아파트여부'] = (feature['임대건물구분'] == '아파트') * 1
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')

feature = pd.merge(feature, age, how = 'left', on = '지역')

feature['공가비율'] = feature['공가수'] / feature['총세대수']

feature['등록차량수'] = (feature['등록차량수'] - feature['단지내주차면수'])

age = ["10대미만"] + [str(i) + '대' for i in range(10, 110, 10)]
for i in age:
  feature[i] = feature[i + "(여자)"] + feature[i + "(남자)"]

train_target = np.array(feature['등록차량수'] / feature['단지내주차면수'])#.astype(int)


#feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
#train_feature = np.array(feature[feature_names_])

train_feature = feature.drop(['등록차량수'], axis = 1)

[Backward Elimination](https://zephyrus1111.tistory.com/65)

In [31]:
# Categorical & '-' columns dropped

train_feature = train_feature.drop(['임대건물구분', '지역', '공급유형', '자격유형', '임대보증금', '임대료'], axis = 1)

In [33]:
## 후진 소거법
variables = train_feature.columns.tolist() ## 설명 변수 리스트
 
y = train_target ## 반응 변수
selected_variables = variables ## 초기에는 모든 변수가 선택된 상태
sl_remove = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(selected_variables) > 0:
    X = sm.add_constant(train_feature[selected_variables])
    p_vals = sm.OLS(y,X.astype(float)).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
    max_pval = p_vals.max() ## 최대 p-value
    if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
 
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(train_feature[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.7/dist-packages/s

In [34]:
selected_variables

['총세대수',
 '전용면적',
 '전용면적별세대수',
 '공가수',
 '단지내주차면수',
 '주거세대수',
 '면적10',
 '면적30',
 '면적40',
 '면적110',
 '경상북도여부',
 '경기도여부',
 '광주광역시여부',
 '충청북도여부',
 '대구광역시여부',
 '80대(남자)',
 '90대(여자)']

# Modeling

## ① Backward Elimination

In [52]:
olsmod = sm.OLS(train_target, train_feature[selected_variables])
olsres = olsmod.fit()

In [43]:
data = test

data1 = data.copy().fillna(0)
feature = data1.drop_duplicates('단지코드').set_index('단지코드')
feature['주거세대수'] = feature['총세대수'] - feature['공가수']

data2 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data2['총전용면적'] = data2['전용면적'] * data2['전용면적별세대수']
data2 = data2.groupby('단지코드')['총전용면적'].sum().reset_index().set_index('단지코드')

data3 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data3['전용면적'] = np.round(data3['전용면적'], -1).astype(int)
list2 = []
for i in range(10, 610, 10):
  list2.append('면적' + str(i))
  data3['면적' + str(i)] = (data3['전용면적'] == i) * data3['전용면적별세대수']
data3 = data3.groupby('단지코드')[list2].sum().reset_index().set_index('단지코드')

feature = pd.merge(feature, data2, on = '단지코드')
feature = pd.merge(feature, data3, on = '단지코드')
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수']
list1 = []
for i in train['지역'].unique():
  list1.append(i + '여부')
  feature[i + '여부'] = (feature['지역'] == i) * 1

feature['아파트여부'] = (feature['임대건물구분'] == '아파트') * 1
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')

feature = pd.merge(feature, age, how = 'left', on = '지역')

age = ["10대미만"] + [str(i) + '대' for i in range(10, 110, 10)]
for i in age:
  feature[i] = feature[i + "(여자)"] + feature[i + "(남자)"]

# feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
feature_names_ = selected_variables
test_feature = np.array(feature[feature_names_])

In [54]:
tmp = feature['단지내주차면수']
pred = olsres.predict(test_feature)


submission['num'] = np.round(tmp * pred + tmp)
submission.to_csv('/content/gdrive/My Drive/Colab Notebooks/LinearReg-FeatureSelected1.csv', index = False)

## ② Backward Elimination + Ridge

In [55]:
from sklearn.linear_model import Ridge

ridgeReg = Ridge(alpha = 0.1, normalize = False)
ridgeReg.fit(train_feature[selected_variables], train_target)
predRidge = ridgeReg.predict(test_feature)


submission['num'] = np.round(tmp * predRidge + tmp)
submission.to_csv('/content/gdrive/My Drive/Colab Notebooks/LinearReg-FeatureSelected1+Ridge.csv', index = False)

## ③ Lasso

In [77]:
from sklearn.linear_model import Lasso
lassoReg = Lasso(alpha = 0.3, normalize = False)
lassoReg.fit(train_feature.drop(['공가비율'], axis = 1), train_target)
predLasso = lassoReg.predict(feature.drop(['임대건물구분', '지역', '공급유형', '자격유형', '임대료', '임대보증금'], axis = 1))

submission['num'] = np.round(tmp * predLasso + tmp)
submission.to_csv('/content/gdrive/My Drive/Colab Notebooks/LinearReg-Lasso.csv', index = False)